In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import scipy.stats as sstats
from sklearn.model_selection import train_test_split
from sklearn.cluster import Birch
from sklearn.metrics import confusion_matrix
from sklearn import (datasets, decomposition, ensemble, manifold, random_projection)
from sklearn.naive_bayes import MultinomialNB

In [2]:
def accuracy_plot(axs, accs):
    plt.figure()
    plt.plot(axs, accs)
    
def KNN(train_X, train_Y, test_X, ks, verbose=False):
    """
    Compute predictions for various k
    Args:
        train_X: array of shape Ntrain x D
        train_Y: array of shape Ntrain
        test_X: array of shape Ntest x D
        ks: list of integers
    Returns:
        preds: dict k: predictions for k
    """
    # Cats data to float32
    train_X = train_X.astype(np.float32)
    test_X = test_X.astype(np.float32)

    # Alloc space for results
    preds = {}
    

    if verbose:
        print("Computing distances... ", end='')
    
    dists = -2 * np.dot(train_X, test_X.T) + np.sum(test_X**2, axis=1) + np.sum(train_X**2, axis=1)[:, np.newaxis]
    
    if verbose:
        print("Sorting... ", end='')

    closest = np.argsort(dists, axis=0)

    if verbose:
        print("Computing predictions...", end='')
    
    targets = train_Y[closest]

    for k in ks:
        predictions = sstats.mode(targets[:k])[0]
        predictions = predictions.ravel()
        preds[k] = predictions
    if verbose:
        print("Done")
    return preds

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

data_batch_1 = unpickle('cifar10/data_batch_1')


db1_labels = np.array(data_batch_1[b'labels'])
db1_data = np.array(data_batch_1[b'data'])

trainx, testx, trainy, testy = train_test_split(db1_data, db1_labels, test_size=0.33)

In [4]:
def kpca():
    print("KPCA")
    OS = np.arange(10,33,3)

    accs = []
    for oss in OS:
        kpca = decomposition.KernelPCA(n_components=oss, kernel="rbf", gamma=10)
        x_proj = kpca.fit_transform(trainx)
        x_test_proj = kpca.transform(testx)
        knn = KNN(x_proj, trainy, x_test_proj, [7])

    #     sum_up(knn[7], testy)
        temp_acc = np.round(np.count_nonzero(knn[7] == testy)/len(testy),2)*100
        accs.append(temp_acc)
        print(f"KNN accuracy: {temp_acc} % on {oss} axes")

    accuracy_plot(OS, accs)
    
kpca()

KPCA


MemoryError: Unable to allocate array with shape (6700, 6700) and data type float64